### Installing & Importing all the necessary packages

Update system packages, install `libgl1`, and install the `openpyxl` library for handling Excel files

- **`albumentations`**: Library for image augmentation to enhance training data diversity.
- **`sweetviz`**: Generates high-density visualizations of pandas DataFrames for quick data analysis.
- **`grad-cam`**: Visualizes important image regions for CNN predictions using Grad-CAM.
- **`lime`**: Provides local explanations for machine learning model predictions.
- **`pandas_profiling`**: Creates detailed reports of pandas DataFrames for exploratory data analysis.
- **`shap`**: Explains model predictions by attributing feature contributions using Shapley values.
- **`Keras-Preprocessing`**: Offers utilities for preprocessing data, including image and text transformations.

In [ ]:
from IPython.display import clear_output

!sudo apt-get update && apt-get install libgl1 -y
!pip install openpyxl -q
!pip install albumentations sweetviz grad-cam lime pandas_profiling shap Keras-Preprocessing -q

clear_output()

In [ ]:
# Standard Libraries
import os
import math
import shutil
import warnings
from typing import Dict, Optional

# Data Handling
import pandas as pd
import numpy as np
import cv2
from PIL import Image

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap

# Skimage
from skimage.segmentation import slic, mark_boundaries

# TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.applications import ResNet50V2, ResNet101V2, InceptionV3, InceptionResNetV2, MobileNetV2, DenseNet169, NASNetMobile, EfficientNetB7, ConvNeXtBase
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, LearningRateScheduler

# Keras (Standalone)
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop, Nadam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau, CSVLogger, LearningRateScheduler

# Machine Learning
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (accuracy_score, precision_score, recall_score, classification_report,
                             confusion_matrix, roc_curve, roc_auc_score, matthews_corrcoef, hamming_loss, f1_score, precision_recall_curve)

# Lime and Shap
from lime import lime_image
import shap

# PyTorch Grad-CAM
from pytorch_grad_cam import GradCAM

# Sweetviz (for EDA)
import sweetviz

# IPython for display
from IPython.display import Image, display

# Magic commands (for Jupyter Notebooks)
%matplotlib inline
%load_ext tensorboard
%reload_ext tensorboard

### Data Loading and Cleaning

Load class labels from an Excel file, binary test labels from a CSV, clean the class labels DataFrame by removing empty rows and columns, and display the cleaned data.

In [ ]:
la1=pd.read_excel('/workspace/anushka saini/train_val/multilabelpcos.xlsx')
la1

,ImagePath,Round and Thin,Cumulus oophorous,Corpus luteum,Hemorrhagic ovarian cyst,Hemorrhagic corpus luteum,Endometrioma,serous cystadenoma,Serous cystadenocarcinoma,Mucinous cystadenoma,...,Foetus,Chocolate cyst,Cervix,Urinary bladder,Polyp,Cervical cyst,Adnexa,Vagina,Uterus,Ovary
0,100image65.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,100image71.jpg,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100image83.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,100image87.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,100image94.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292,pco_3.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2293,pco_5.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2294,pco_6.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2295,pco_7.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
la2 = pd.read_csv('/workspace/anushka saini/test/test_label_multi.csv')
la2

,imagePath,Round and Thin,Cumulus oophorous,Corpus luteum,Hemorrhagic ovarian cyst,Hemorrhagic corpus luteum,Endometrioma,serous cystadenoma,Serous cystadenocarcinoma,Mucinous cystadenoma,...,Thick hyperechoic margin,Vaginal ultrasound,Transvaginal ultrasound,Gestational sac,Foetus,Chocolate cyst,Cervix,Urinary bladder,Polyp,Cervical cyst
0,image10000.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,image10001.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,image10002.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,image10003.jpg,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,image10004.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,image11463.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1464,image11464.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1465,image11465.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1466,image11466.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = la1.dropna(how='all').dropna(how='all', axis=1)
print(df)

           ImagePath  Round and Thin  Cumulus oophorous  Corpus luteum  \
0     100image65.jpg               0                  0              0   
1     100image71.jpg               0                  0              0   
2     100image83.jpg               0                  0              0   
3     100image87.jpg               0                  0              0   
4     100image94.jpg               0                  0              0   
...              ...             ...                ...            ...   
2292       pco_3.jpg               0                  0              0   
2293       pco_5.jpg               0                  0              0   
2294       pco_6.jpg               0                  0              0   
2295       pco_7.jpg               0                  0              0   
2296       pco_8.jpg               0                  0              0   

      Hemorrhagic ovarian cyst  Hemorrhagic corpus luteum  Endometrioma  \
0                            0      

### Train-Validate Split

Split the DataFrame into training and validation sets based on specified percentages, using optional random seed for reproducibility.

In [ ]:
def train_validate_split(df, train_percent=.8, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    # test = df.iloc[perm[validate_end:]]
    return train, validate

In [ ]:
train, validate = train_validate_split(df)
train

,ImagePath,Round and Thin,Cumulus oophorous,Corpus luteum,Hemorrhagic ovarian cyst,Hemorrhagic corpus luteum,Endometrioma,serous cystadenoma,Serous cystadenocarcinoma,Mucinous cystadenoma,...,Foetus,Chocolate cyst,Cervix,Urinary bladder,Polyp,Cervical cyst,Adnexa,Vagina,Uterus,Ovary
2205,image3742.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
543,image0252.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,104image0086.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
178,202image106.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1621,image2260.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,image0581.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1809,image2722.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1141,image0957.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
370,89image0098.jpg,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
validate

,ImagePath,Round and Thin,Cumulus oophorous,Corpus luteum,Hemorrhagic ovarian cyst,Hemorrhagic corpus luteum,Endometrioma,serous cystadenoma,Serous cystadenocarcinoma,Mucinous cystadenoma,...,Foetus,Chocolate cyst,Cervix,Urinary bladder,Polyp,Cervical cyst,Adnexa,Vagina,Uterus,Ovary
951,image0724.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415,95image93.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
290,523image18.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1279,image1117.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,32image0025.jpg,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,image3769.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1666,image2417.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1690,image2467.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1878,image3115.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Image and Label Preparation

Map labels, load and preprocess images for training, validation, and testing, and display shapes of image arrays and labels.

In [ ]:
train_labels = train[["Round and Thin", "Cumulus oophorous", "Corpus luteum", "Hemorrhagic ovarian cyst", "Hemorrhagic corpus luteum", "Endometrioma", "serous cystadenoma", "Serous cystadenocarcinoma", "Mucinous cystadenoma", "Mucinous cystadenocarcinoma", "Dermoid cyst", "Dermoid plug", "Rokitansky nodule", "Dermoid mesh", "Dot dash pattern", "Floating balls sign", "Ovarian fibroma", "Ovarian thecoma", "Metastasis", "Para ovarian cyst", "Polycystic ovary", "Ovarian hyperstimulation syndrome", "Ovarian torsion", "Thick hyperechoic margin", "Vaginal ultrasound", "Transvaginal ultrasound", "Gestational sac", "Foetus", "Chocolate cyst", "Cervix", "Urinary bladder", "Polyp", "Cervical cyst"]].values
train_image_paths = [os.path.join('/workspace/anushka saini/train_val/images', filename) for filename in train['ImagePath']] #to change when using GPU. similarly for validate and test
train_images = []
for train_image_path in train_image_paths:
    image = load_img(train_image_path, target_size=(300, 300)) # change this when you run on GPU. 320X320 rahegi
    image = img_to_array(image) / 255.0  # Normalize pixel values between 0 and 1
    train_images.append(image)
train_images = np.array(train_images, dtype=np.float32)

In [ ]:
train_images.shape

(1837, 300, 300, 3)

In [ ]:
train_labels.shape

(1837, 33)

In [ ]:
validate_labels = validate[["Round and Thin", "Cumulus oophorous", "Corpus luteum", "Hemorrhagic ovarian cyst", "Hemorrhagic corpus luteum", "Endometrioma", "serous cystadenoma", "Serous cystadenocarcinoma", "Mucinous cystadenoma", "Mucinous cystadenocarcinoma", "Dermoid cyst", "Dermoid plug", "Rokitansky nodule", "Dermoid mesh", "Dot dash pattern", "Floating balls sign", "Ovarian fibroma", "Ovarian thecoma", "Metastasis", "Para ovarian cyst", "Polycystic ovary", "Ovarian hyperstimulation syndrome", "Ovarian torsion", "Thick hyperechoic margin", "Vaginal ultrasound", "Transvaginal ultrasound", "Gestational sac", "Foetus", "Chocolate cyst", "Cervix", "Urinary bladder", "Polyp", "Cervical cyst"]].values
validate_image_paths = [os.path.join('/workspace/anushka saini/train_val/images', filename) for filename in validate['ImagePath']]
validate_images = []
for validate_image_path in validate_image_paths:
    image = load_img(validate_image_path, target_size=(300, 300)) # change this when you run on GPU
    image = img_to_array(image) / 255.0  # Normalize pixel values between 0 and 1
    validate_images.append(image)
validate_images = np.array(validate_images, dtype=np.float32)

In [ ]:
validate_images.shape

(459, 300, 300, 3)

In [ ]:
validate_labels.shape

(459, 33)

In [ ]:
test_labels = la2[["Round and Thin", "Cumulus oophorous", "Corpus luteum", "Hemorrhagic ovarian cyst", "Hemorrhagic corpus luteum", "Endometrioma", "serous cystadenoma", "Serous cystadenocarcinoma", "Mucinous cystadenoma", "Mucinous cystadenocarcinoma", "Dermoid cyst", "Dermoid plug", "Rokitansky nodule", "Dermoid mesh", "Dot dash pattern", "Floating balls sign", "Ovarian fibroma", "Ovarian thecoma", "Metastasis", "Para ovarian cyst", "Polycystic ovary", "Ovarian hyperstimulation syndrome", "Ovarian torsion", "Thick hyperechoic margin", "Vaginal ultrasound", "Transvaginal ultrasound", "Gestational sac", "Foetus", "Chocolate cyst", "Cervix", "Urinary bladder", "Polyp", "Cervical cyst"]].values
test_image_paths = [os.path.join('/workspace/anushka saini/test/images', filename) for filename in la2['imagePath']]
test_images = []
for test_image_path in test_image_paths:
    image = load_img(test_image_path, target_size=(300, 300)) # change this when you run on GPU
    image = img_to_array(image) / 255.0  # Normalize pixel values between 0 and 1
    test_images.append(image)
test_images = np.array(test_images, dtype=np.float32)

In [ ]:
test_images.shape

(1468, 300, 300, 3)

In [ ]:
test_labels.shape

(1468, 33)

In [25]:
base_model=EfficientNetB7(weights='imagenet', include_top=False,input_shape=(300,300,3))

2024-01-31 05:34:10.197143: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-31 05:34:10.299233: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1621] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38318 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0


258076736/258076736 [==============================] - 18s 0us/step


In [26]:
x = base_model.output
x = Flatten()(x)
predictions = Dense(33, activation='sigmoid',
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,)(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'binary_accuracy', 'categorical_accuracy', 'top_k_categorical_accuracy', 'Precision', 'Recall'])
csv_logger = CSVLogger("EfficientNetB7_model_history_log.csv", append=True) #to be changed as per model

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 300, 300, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 300, 300, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 300, 300, 3)  0           ['normalization[0][0]']      

### Training, Saving, and Evaluating a EfficientNetB7 Model

Trains the EfficientNetB7 model, saves and reloads weights, and evaluates performance on validation and test datasets using classification reports.

In [32]:
model.fit(train_images,train_labels,validation_data=(validate_images,validate_labels),batch_size=32,
          epochs=250,callbacks=csv_logger,shuffle=False,class_weight=None,steps_per_epoch=None,
          validation_steps=None,
          validation_batch_size=None,
          validation_freq=1,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False) #make it 250

Epoch 1/250


2024-01-31 05:35:11.184537: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700
2024-01-31 05:35:11.312386: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:648] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


58/58 [==============================] - 27s 220ms/step - loss: 1.5247 - accuracy: 0.2254 - binary_accuracy: 0.9751 - categorical_accuracy: 0.2254 - top_k_categorical_accuracy: 0.9630 - precision: 0.0507 - recall: 0.0984 - val_loss: 0.9045 - val_accuracy: 0.1634 - val_binary_accuracy: 0.9701 - val_categorical_accuracy: 0.1634 - val_top_k_categorical_accuracy: 0.2113 - val_precision: 0.1634 - val_recall: 0.5208
Epoch 2/250
58/58 [==============================] - 10s 168ms/step - loss: 0.9985 - accuracy: 0.0724 - binary_accuracy: 0.9840 - categorical_accuracy: 0.0724 - top_k_categorical_accuracy: 0.8280 - precision: 0.0940 - recall: 0.0893 - val_loss: 0.6207 - val_accuracy: 0.0174 - val_binary_accuracy: 0.9905 - val_categorical_accuracy: 0.0174 - val_top_k_categorical_accuracy: 0.2244 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/250
58/58 [==============================] - 10s 173ms/step - loss: 0.9308 - accuracy: 0.0925 - binary_accuracy: 0.9834 - categorical_accuracy: 

In [ ]:
model.save_weights('EfficientNetB7_weights.h5')
model.save('EfficientNetB7_model.h5')
model.load_weights('EfficientNetB7_weights.h5')

In [36]:
validate_labels_pred=model.predict(validate_images)
validate_labels_pred=np.argmax(validate_labels_pred,axis=1)
validate_labels=np.argmax(validate_labels,axis=1)

15/15 [==============================] - 5s 114ms/step


In [37]:
print(classification_report(validate_labels,validate_labels_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       328
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         8
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         8
           6       0.00      0.00      0.00         5
          10       0.00      0.00      0.00         2
          20       0.16      1.00      0.28        75
          21       1.00      0.07      0.13        14
          25       0.00      0.00      0.00         1
          26       0.00      0.00      0.00         4
          30       0.00      0.00      0.00         1

    accuracy                           0.17       459
   macro avg       0.09      0.08      0.03       459
weighted avg       0.06      0.17      0.05       459



In [38]:
test_labels_pred=model.predict(test_images)
test_labels_pred=np.argmax(test_labels_pred,axis=1)
test_labels=np.argmax(test_labels,axis=1)

46/46 [==============================] - 5s 112ms/step


In [39]:
print(classification_report(test_labels,test_labels_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       512
           1       0.00      0.00      0.00       194
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00        31
          10       0.00      0.00      0.00        28
          16       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          19       0.00      0.00      0.00         1
          20       0.37      1.00      0.54       549
          21       0.00      0.00      0.00         0
          24       0.00      0.00      0.00         3
          25       0.00      0.00      0.00        35
          26       0.00      0.00      0.00         4
          28       0.00      0.00      0.00         1
          30       0.00      0.00      0.00         7
          31       0.00      0.00      0.00        84

    accuracy              